In [1]:
from ultralytics import YOLO
import cv2, numpy as np, matplotlib.pyplot as plt, os

In [2]:
def pad_resize_bbox(img, xyxy, out_size=300, pad_color=(255, 255, 255)):
    """给定原图和 bbox，返回等比例放缩+白底居中后的 out_size×out_size 图像"""
    x1, y1, x2, y2 = xyxy
    crop = img[y1:y2, x1:x2]
    h, w = crop.shape[:2]

    scale = out_size / max(w, h)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(crop, (new_w, new_h), interpolation=cv2.INTER_AREA)

    canvas = np.full((out_size, out_size, 3), pad_color, dtype=np.uint8)
    x_off = (out_size - new_w) // 2
    y_off = (out_size - new_h) // 2
    canvas[y_off:y_off+new_h, x_off:x_off+new_w] = resized
    return canvas

In [3]:
def extract_bbox(img_path, model):
    img = cv2.imread(img_path)
    res  = model.predict(img, conf=0.25, save=False)[0]

    if len(res.boxes) == 0:
        print(f"{img_path} No cats detected!")

    # 取置信度最高的 bbox
    xyxy = res.boxes.xyxy[res.boxes.conf.argmax()].cpu().numpy().astype(int)
    x1, y1, x2, y2 = xyxy

    # 2️⃣ 生成 300×300 居中白底缩放图
    padded = pad_resize_bbox(img, xyxy, out_size=300)

    # 3️⃣ 画图展示
    plt.figure(figsize=(8,4))
    # 原图 + bbox
    plt.subplot(1,2,1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.gca().add_patch(plt.Rectangle((x1,y1), x2-x1, y2-y1, fill=False, edgecolor='red', linewidth=2))
    plt.title(f"Original {i}")
    plt.axis('off')
    # 处理后 300x300
    plt.subplot(1,2,2)
    plt.imshow(cv2.cvtColor(padded, cv2.COLOR_BGR2RGB))
    plt.title("Padded 300×300")
    plt.axis('off')
    plt.tight_layout()
    plt.show()